In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from subprocess import call
from scipy.stats.distributions import gamma, lognorm
import json
import wget
import os
import os.path
from datetime import datetime
import pytz
from collections import OrderedDict
#from google.colab import drive
import math

from datetime import datetime, timedelta
from urllib.request import urlopen
import collections
import copy
#os.chdir('/content/gdrive/My Drive')
#drive.mount('/content/gdrive')

In [2]:
district_json = json.load(open('allmetrics_districts.json'))
states_json = json.load(open('allmetrics_states.json'))


# delhi = {}

# delhi['Delhi'] = states['Delhi']
# delhi['Chandigarh'] = states['Chandigarh']

# delhi['datetime'] = states['datetime']

# with open('delhi-demo.json','w') as file:
#     json.dump(delhi,file)

# districts["Ahmednagar"]

In [3]:
# states = json.load(open('delhi-demo.json'))

districts = list(district_json.keys())
districts.remove("datetime")
# districts

In [4]:
state_population=pd.read_csv('population.csv')

population = pd.DataFrame()
population["district"] = districts
population.set_index("district",inplace=True)
population["Population"] = 5
population

,Population
district,
Anantapur,5
Chittoor,5
East Godavari,5
Guntur,5
Kurnool,5
...,...
Lucknow,5
Varanasi,5
Prayagraj,5


In [5]:
def update_icu(districts):
    for i,w in enumerate(districts["pct_ICU"]):
        districts["pct_ICU"][i] = 15
    return districts
#     may11_index = 467
#     may12_index = 468
#     may15_index = 471
#     may16_index = 472
    
#     if dist == "Delhi":
#         states[dist]["pct_ICU"][may12_index] = 1.3
#         states[dist]["pct_ICU"][may15_index] = 11.4
#         states[dist]["pct_ICU"][may16_index] = 11.4
#     elif dist=="Chandigarh":
#         states[dist]["pct_ICU"][may11_index] = 7.3
#         states[dist]["pct_ICU"][may12_index] = 11.2
#         states[dist]["pct_ICU"][may15_index] = 14.9
#         states[dist]["pct_ICU"][may16_index] = 14.9

def convert_vacc(dat):
    try:
        return datetime.strptime(dat, '%d/%m/%Y').strftime('%d %B')
    except:
        return datetime.strptime(dat, '%d-%m-%Y').strftime('%d %B')


def get_daily_series(a):
    return [t - s for s, t in zip(a, a[1:])]


'''
def moving_avg(data, window_size=7):
    numbers_series = pd.Series(data)
    windows = numbers_series.rolling(window_size)
    moving_averages = windows.mean()
    moving_averages_list = moving_averages.tolist()
    return moving_averages_list[window_size - 1:]
'''

def moving_avg(data, window_size=7):
    ma = ['']*len(data)
    for w in range(window_size, len(data)):
        sum1 = 0
        count = 0
        for s in range(window_size):
            if (len(str(data[w - s])) != 0):
                sum1 += int(data[w - s])
                count += 1
        if count != 0:
            ma[w] = sum1 / count
    
    return ma


def rate_increase_cases(a, window_size=14):
    increase_rate = ['']*window_size
    
    for t,s in zip(a[window_size:],a):
        try:
            increase_rate.append((t - s) * 100 / t)
        except:
            increase_rate.append('')
            
    
    return increase_rate


def TRP_14ma(daily_pos, daily_tested, window_size=7):
    daily_pos_rate_ma = ['']*len(daily_pos)
    for w in range(window_size, len(daily_pos)):
        sum1 = 0
        sum2 = 0
        for s in range(window_size):
            if (len(str(daily_pos[w - s])) != 0
                    and len(str(daily_tested[w - s])) != 0):
                sum1 += int(daily_pos[w - s])
                sum2 += int(daily_tested[w - s])

        if sum2 != 0:
            daily_pos_rate_ma[w] = sum1 * 100 / abs(sum2)

    return daily_pos_rate_ma


def category_change(category):
    return [t != s for s, t in zip(category, category[1:])]

def divide_helper(a,b):
    replace = {'': 0}
    restore = {0:''}
    
    a = [replace.get(x, x) for x in a]
    a = np.array(a)*1000000/b
    
    return [restore.get(x, x) for x in a]

In [6]:
def logic(js,i):
    if js["rate_increase_new_cases"][i] == '' or js["TPR"][i] == '' or js["pct_ICU"][i] == '' or js["daily_cases_per_million"][i] == '' or js["daily_tests_per_million_14ma"][i] == '':
        return "GRAY"
    elif js["daily_cases_per_million"][i]  >100 and js["TPR"][i] > 10 and js["pct_ICU"][i] < 40 and (js["rate_increase_new_cases"][i] >5  or js["daily_tests_per_million_14ma"][i] < 140):
        return "RED"
    elif (js["daily_cases_per_million"][i]  >= 20) and (js["TPR"][i] >=5) and (js["pct_ICU"][i] <= 80) and ((js["rate_increase_new_cases"][i] >= 2) or (js["daily_tests_per_million_14ma"][i] < 140)):
        return "YELLOW"
    else:
        return "GREEN"

    
def agg_logic(js,i):
    if js["rate_increase_new_cases"][i] == '' or js["TPR"][i] == '' or js["pct_ICU"][i] == '' or js["daily_cases_per_million"][i] == '' or js["daily_tests_per_million_14ma"][i] == '':
        return "GRAY"
    if js["daily_cases_per_million"][i]  <20 and js["TPR"][i] < 5 and js["pct_ICU"][i] > 80 and (js["rate_increase_new_cases"][i] < 2 or js["daily_tests_per_million_14ma"][i] > 140):
        return "GREEN"
    elif (js["daily_cases_per_million"][i]  <100) and (js["TPR"][i] <= 10) and js["pct_ICU"][i] >= 40 and  ((js["rate_increase_new_cases"][i] <=5) or (js["daily_tests_per_million_14ma"][i] < 140)):
        return "YELLOW"
    else:
        return "RED"

def logic_without_icu(js,i):
    if js["rate_increase_new_cases"][i] == '' or js["TPR"][i] == '' or js["daily_cases_per_million"][i] == '' or js["daily_tests_per_million_14ma"][i] == '':
        return "GRAY"
    elif js["daily_cases_per_million"][i]  >100 and js["TPR"][i] > 10 and ( js["rate_increase_new_cases"][i] >5 or js["daily_tests_per_million_14ma"][i] < 140):
        return "RED"
    elif (js["daily_cases_per_million"][i]  >= 20) and (js["TPR"][i] >=5) and ((js["rate_increase_new_cases"][i] >= 2) or (js["daily_tests_per_million_14ma"][i] < 140)):
        return "YELLOW"
    else:
        return "GREEN"
    
    
def agg_logic_without_icu(js,i):
    if js["rate_increase_new_cases"][i] == '' or js["TPR"][i] == '' or js["daily_cases_per_million"][i] == '' or js["daily_tests_per_million_14ma"][i] == '':
        return "GRAY"
    elif js["daily_cases_per_million"][i]  <20 and js["TPR"][i] < 5 and (js["rate_increase_new_cases"][i] < 2 or js["daily_tests_per_million_14ma"][i] > 140):
        return "GREEN"
    elif (js["daily_cases_per_million"][i]  <100) and (js["TPR"][i] <= 10) and ((js["rate_increase_new_cases"][i] <=5) or (js["daily_tests_per_million_14ma"][i] < 140)):
        return "YELLOW"
    else:
        return "RED"
    
def yellow_logic_without_icu(js,i):
    if js["rate_increase_new_cases"][i] == '' or js["TPR"][i] == '' or js["daily_cases_per_million"][i] == '' or js["daily_tests_per_million_14ma"][i] == '':
        return "GRAY"
    elif js["daily_cases_per_million"][i]  <20 and js["TPR"][i] < 5 and (js["rate_increase_new_cases"][i] < 2 or js["daily_tests_per_million_14ma"][i] > 140):
        return "GREEN"
    elif js["daily_cases_per_million"][i]  >100 and js["TPR"][i] > 10 and ( js["rate_increase_new_cases"][i] >5 or js["daily_tests_per_million_14ma"][i] < 140):
        return "RED"
    else:
        return "YELLOW"

In [7]:
t = ['','','','','',3,2,6,2,7,2,7]

rate_increase_cases(t,3)

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 -50.0,
 71.42857142857143,
 -200.0,
 71.42857142857143]

In [8]:
for dist in districts:

    daily_pos = district_json[dist]['daily_positive_cases']
    daily_positive_cases_14ma = moving_avg(daily_pos,14)
    district_json[dist]['daily_positive_cases_14ma'] = daily_positive_cases_14ma
    
    daily_tested = district_json[dist]['daily_tests']
    daily_tests_14ma = moving_avg(daily_tested,14)
    district_json[dist]['daily_tests_14ma'] = daily_tests_14ma
    
    district_json[dist]['rate_increase_new_cases'] = rate_increase_cases(daily_positive_cases_14ma,14)
    
    district_json[dist]['TPR'] = TRP_14ma(daily_pos,daily_tested,14)
    
    district_json[dist]['daily_tests_per_million_14ma'] = divide_helper(daily_tests_14ma,int(population['Population'][dist]))
    district_json[dist]['pct_ICU'] = [''] * len(district_json[dist]['dates'])  #TODO
    district_json[dist] = update_icu(district_json[dist])
        
    district_json[dist]['category'] = [logic_without_icu(district_json[dist], i) for i in range(len(district_json[dist]['dates']))] #TODO - outputted by some function
    
    
    
    district_json[dist]['year'] = ['']
    
    district_json[dist]['is_change'] = ['']+category_change(district_json[dist]['category']) #TODO - Bool value: 1 when category changed from previous date or not
    

TypeError: list indices must be integers or slices, not str

In [ ]:
with open('demo.json','w') as file:
    json.dump(district_json,file)

In [ ]:
for dist in districts:
    
    daily_pos = district_json[dist]['daily_positive_cases']
    daily_positive_cases_14ma = moving_avg(daily_pos,14)
    district_json[dist]['daily_positive_cases_14ma'] = daily_positive_cases_14ma
    
    daily_tested = district_json[dist]['daily_tests']
    daily_tests_14ma = moving_avg(daily_tested,14)
    district_json[dist]['daily_tests_14ma'] = daily_tests_14ma
    
    district_json[dist]['rate_increase_new_cases'] = rate_increase_cases(daily_positive_cases_14ma,14)
    
    district_json[dist]['TPR'] = TRP_14ma(daily_pos,daily_tested,14)
    
    district_json[dist]['daily_tests_per_million_14ma'] = divide_helper(daily_tests_14ma,int(population['Population'][dist]))
    district_json[dist]['pct_ICU'] = [''] * len(district_json[dist]['dates'])  #TODO
    district_json[dist] = update_icu(district_json[dist])
        
    district_json[dist]['category'] = [logic(district_json[dist], i) for i in range(len(district_json[dist]['dates']))] #TODO - outputted by some function
    
    
    
    district_json[dist]['year'] = ['']
    
    district_json[dist]['is_change'] = ['']+category_change(district_json[dist]['category']) #TODO - Bool value: 1 when category changed from previous date or not
    

In [ ]:
with open('actual.json','w') as file:
    json.dump(district_json,file)
    

In [ ]:
for dist in districts:
    
    daily_pos = district_json[dist]['daily_positive_cases']
    daily_positive_cases_14ma = moving_avg(daily_pos,14)
    district_json[dist]['daily_positive_cases_14ma'] = daily_positive_cases_14ma
    
    daily_tested = district_json[dist]['daily_tests']
    daily_tests_14ma = moving_avg(daily_tested,14)
    district_json[dist]['daily_tests_14ma'] = daily_tests_14ma
    
    district_json[dist]['rate_increase_new_cases'] = rate_increase_cases(daily_positive_cases_14ma,14)
    
    district_json[dist]['TPR'] = TRP_14ma(daily_pos,daily_tested,14)
    
    district_json[dist]['daily_tests_per_million_14ma'] = divide_helper(daily_tests_14ma,int(population['Population'][dist]))
    district_json[dist]['pct_ICU'] = ['']*len(district_json[dist]['dates']) #TODO
    district_json[dist] = update_icu(district_json[dist])
    
    district_json[dist]['category'] = [agg_logic_without_icu(district_json[dist], i) for i in range(len(district_json[dist]['dates']))] #TODO - outputted by some function
    district_json[dist]['year'] = ['']*len(district_json[dist]['dates'])
    

    
    district_json[dist]['is_change'] = ['']+category_change(district_json[dist]['category']) #TODO - Bool value: 1 when category changed from previous date or not
    
    

In [ ]:
with open('agg_demo.json','w') as file:
    json.dump(district_json,file)
    

In [ ]:
for dist in districts:
    
    daily_pos = district_json[dist]['daily_positive_cases']
    daily_positive_cases_14ma = moving_avg(daily_pos,14)
    district_json[dist]['daily_positive_cases_14ma'] = daily_positive_cases_14ma
    
    daily_tested = district_json[dist]['daily_tests']
    daily_tests_14ma = moving_avg(daily_tested,14)
    district_json[dist]['daily_tests_14ma'] = daily_tests_14ma
    
    district_json[dist]['rate_increase_new_cases'] = rate_increase_cases(daily_positive_cases_14ma,14)
    
    district_json[dist]['TPR'] = TRP_14ma(daily_pos,daily_tested,14)
    
    district_json[dist]['daily_tests_per_million_14ma'] = divide_helper(daily_tests_14ma,int(population['Population'][dist]))
    district_json[dist]['pct_ICU'] = ['']*len(district_json[dist]['dates']) #TODO
    
    district_json[dist] = update_icu(district_json[dist])    
    district_json[dist]['category'] = [agg_logic(district_json[dist], i) for i in range(len(district_json[dist]['dates']))] #TODO - outputted by some function
    district_json[dist]['year'] = ['']*len(district_json[dist]['dates'])
    

    
    district_json[dist]['is_change'] = ['']+category_change(district_json[dist]['category']) #TODO - Bool value: 1 when category changed from previous date or not
    

In [ ]:
with open('agg_actual.json','w') as file:
    json.dump(district_json,file)
    
    

In [ ]:
for dist in districts:
    
    daily_pos = district_json[dist]['daily_positive_cases']
    daily_positive_cases_14ma = moving_avg(daily_pos,14)
    district_json[dist]['daily_positive_cases_14ma'] = daily_positive_cases_14ma
    
    daily_tested = district_json[dist]['daily_tests']
    daily_tests_14ma = moving_avg(daily_tested,14)
    district_json[dist]['daily_tests_14ma'] = daily_tests_14ma
    
    district_json[dist]['rate_increase_new_cases'] = rate_increase_cases(daily_positive_cases_14ma,14)
    
    district_json[dist]['TPR'] = TRP_14ma(daily_pos,daily_tested,14)
    
    district_json[dist]['daily_tests_per_million_14ma'] = divide_helper(daily_tests_14ma,int(population['Population'][dist]))
    district_json[dist]['pct_ICU'] = ['']*len(district_json[dist]['dates']) #TODO
    
    district_json[dist] = update_icu(district_json[dist])    
    district_json[dist]['category'] = [yellow_logic_without_icu(district_json[dist], i) for i in range(len(district_json[dist]['dates']))] #TODO - outputted by some function
    district_json[dist]['year'] = ['']*len(district_json[dist]['dates'])
    

    
    district_json[dist]['is_change'] = ['']+category_change(district_json[dist]['category']) #TODO - Bool value: 1 when category changed from previous date or not
    

In [ ]:
with open('yellow_logic_without_icu.json','w') as file:
    json.dump(district_json,file)
    
    

In [ ]:
# district_json["Anantapur"]["pct_ICU"]

In [ ]:
# districts